In [6]:
import sys
sys.path.append('../')
from utils.notebook_imports import *
from tactic_and_execution.thought_action_observation_data_structure import *
from collections import Counter

# this notebook showcases how to play with the trajectory structure

### load samples

In [18]:
data = jload('../data/traj_toydata.json')

In [19]:
Counter(e['dataset'] for e in data)

Counter({'folio': 7, 'gsm8k': 5, 'reclor': 37, 'proscript': 6})

### a sample has the following fields

In [20]:
Counter([k for e in data for k in e])

Counter({'input': 55,
         'label': 55,
         'src': 55,
         'dataset': 55,
         'orig_data': 55,
         'traj': 51,
         'question_type': 37})

#### The field `sample['traj']` saves the json dump of a Trajectory

In [27]:
sample = data[6] # this is a gsm8k data

In [28]:
# load the traj using from_json method
traj = Trajectory.from_json(sample['traj'])

### you can visualize it by `str` or `nice_str()`

In [29]:
print(traj)

=== init prompt ===


You try to solve the following question with a specific tactic. 
To do so, you will follow the tactic provided below:
- read and understand the tactic and its scope
- perform and ONLY perform actions in **Action space**

You will interact with an external environment from which you will get observations as the results
of your actions. The interaction consists of multiple round of (Thought, Action, Observation), and you will generate
Thought and Action in the following format
"
### Thought
<your thought>

### Action
## Name
<exact name of the action>
## Input
<input of the action>
## Output
<output of the action>
"

For every round you will:
- briefly reflect and generate your Thought about the next action to be taken; fill it under "### Thought"
- perform one Action defined in **Action space** and generate the corresponding output; fill the name, input, 
  and output under "### Action". 
  - For "## Input", if the action takes in the whole history (e.g., code/obse

In [30]:
print(traj.nice_str())

=== init prompt ===



Answer the question below.

### Question:
The cost of purchasing two commodities is $827. If the purchase price of the first one exceeds the other by $127, what is the purchase price of the first commodity?



=== trajectory ===

=== response === status: exec ok

### Thought

This is an arithmetic math problem and can be solved by building a math model and solves it with Python math libs.

### Action
## Name
Plan
## Input
the problem given
## Output

This is the program sketch I'll use to assist my reasoning. 
I should never call Tactic check because this is the right tactic to solve it.

```python
import numpy as np
import sympy
import math

# build the model

# print the output   
```

Finally, I will call Aggregate and answer and return just the numerical number as the answer without further explanation


=== observations ===

# Observer: Echo
# Feedback status: feedback ok
# Content:
OK!

######

=== response === status: exec ok

### Thought

Next, following 

### a traj has init_prompt and exp_ls which is a list of ExpTuple

In [32]:
print(traj.init_prompt)


You try to solve the following question with a specific tactic. 
To do so, you will follow the tactic provided below:
- read and understand the tactic and its scope
- perform and ONLY perform actions in **Action space**

You will interact with an external environment from which you will get observations as the results
of your actions. The interaction consists of multiple round of (Thought, Action, Observation), and you will generate
Thought and Action in the following format
"
### Thought
<your thought>

### Action
## Name
<exact name of the action>
## Input
<input of the action>
## Output
<output of the action>
"

For every round you will:
- briefly reflect and generate your Thought about the next action to be taken; fill it under "### Thought"
- perform one Action defined in **Action space** and generate the corresponding output; fill the name, input, 
  and output under "### Action". 
  - For "## Input", if the action takes in the whole history (e.g., code/observation/thoughts so f

In [34]:
print(len(traj.exp_ls))
# you can get the overview of exp_ls this way
print(traj.show_traj_status())

4
[('Plan', 'exec ok'), ('Build math model', 'exec ok'), ('Aggregate and answer', 'exec wrong'), ('Aggregate and answer', 'exec solved')]


### this means this traj has four steps, Plan -> Build math model -> answer (failed) -> answer (correct)

### for every exp, you can inspect the response, action_type, observation (a list of Feedback), and the exec status

In [35]:
exp = traj.exp_ls[0]

In [36]:
print(exp.response)

### Thought
This is an arithmetic math problem and can be solved by building a math model and solves it with Python math libs.
### Action
## Name
Plan
## Input
the problem given
## Output
This is the program sketch I'll use to assist my reasoning. 
I should never call Tactic check because this is the right tactic to solve it.

```python
import numpy as np
import sympy
import math

# build the model

# print the output   
```

Finally, I will call Aggregate and answer and return just the numerical number as the answer without further explanation


In [37]:
print(exp.action_type)

#A# Plan
- Input: the problem given
- Functionality: give a plan on how to solve the question, including a sketch of the solution, libs to be used,
and code snippets
- Output: text description of the plan and potential code snippets of the form
    ```python
    <your code>
    ```


In [38]:
print(exp.observation)

[Feedback(observer='Echo', content='OK!', feedback_status='feedback ok')]


In [39]:
print(exp.exec_status)

exec ok


# if you have run the `example_traj_datagen.py` and got the results, you can inspect the outcome below

In [49]:
data = jload('../data/traj_toydata-output.json')

In [50]:
Counter([k for e in data for k in e])

Counter({'input': 55,
         'label': 55,
         'src': 55,
         'dataset': 55,
         'orig_data': 55,
         'traj': 51,
         'icl_gpt4_traj': 4})

### now we have additional field `sample['icl_gpt4_traj']` which is the new traj generated by gpt4, we can inspect them the same way as we did above

In [52]:
data2 = [e for e in data if 'icl_gpt4_traj' in e]

In [53]:
# first check the steps
for sample in data2:
    print(Trajectory.from_json(sample['icl_gpt4_traj']).show_traj_status())

[('Plan', 'exec ok'), ('Build math model', 'exec wrong'), ('Revise code', 'exec wrong'), ('Revise code', 'exec ok'), ('Aggregate and answer', 'exec wrong'), ('Aggregate and answer', 'exec solved')]
[(None, 'exec wrong'), ('Plan', 'exec ok'), ('Build FOL model', 'exec ok'), ('Aggregate and answer', 'exec wrong'), ('Aggregate and answer', 'exec solved')]
[('Plan', 'exec ok'), ('Build graph model', 'exec ok'), ('Aggregate and answer', 'exec wrong'), ('Aggregate and answer', 'exec wrong'), ('Aggregate and answer', 'exec solved')]
[('Plan', 'exec ok'), ('Write program', 'exec ok'), ('Aggregate and answer', 'exec wrong'), ('Aggregate and answer', 'exec solved')]


In [54]:
# you check the final status this way
for sample in data2:
    print(Trajectory.from_json(sample['icl_gpt4_traj']).cur_status())

exec solved
exec solved
exec solved
exec solved


In [56]:
# finally you can check the traj just like the above
print(Trajectory.from_json(data2[0]['icl_gpt4_traj']).nice_str())

=== init prompt ===



Answer the question below.

### Question:
Sid traveled 110 miles in 2 hours. If Sid then traveled an additional 140 miles in 3 hours, what's the average speed he was traveling?



=== trajectory ===

=== response === status: exec ok

### Thought
To find the average speed for the entire journey, I need to calculate the total distance covered and total time taken during two segments of Sid's travel. Once I have both totals, I can find the average speed using the formula \( \text{average speed} = \frac{\text{total distance}}{\text{total time}} \). I will start by planning out the approach and the mathematical model to solve this.

### Action
## Name
Plan
## Input
the problem given
## Output
I am planning to solve the calculation of average speed by summing up the distances and times from the two segments of the trip and then applying the formula for average speed. I will use basic arithmetic calculations, and the operation will be executed using Python's built-in ca